<a id='TOC'></a>
# Project: *Analysis of Future of Buffalo Breeds and Milk Production growth in India*

`
By   : Shahapurkar, Gangaprasad
Email: garashah@iu.edu`

<img src="../images/cover_photo.jpg" alt="Cover photo" style="width:=500px;height:=300px;"/>

## Abstract and objective


Water buffalo (**Bubalus bubalis**) also called *Domestic Water Buffalo* or *Asian Water Buffalo* is a large bovid, originating in Indian subcontinent, Southeast Asia, and China. Today, it is also found in Europe, Australia, North America, South America and some African countries. Two extant types of water buffalo are recognized based on morphological and behavioural criteria:

**1. River Buffalo** - Found in Indian subcontinent and further west to the Balkans, Egypt, and Italy

**2. Swamp Buffalo** - Found from Assam in the west through Southeast Asia to the Yangtze valley of China in the east

India has highest level of milk production and consumption of all countries. The Dairy Industry in India is unique among large-scale milk producing countries in terms of its large share of buffalo milk. The aim of this academic project is to study the livestock census data of buffalo breeds in India and their milk production using Empirical Benchmarking analysis method at state level. Looking at the small sample of data, our analysis indicates that we have been seeing increasing trends in past few years in livestock and milk production but there are considerable opportunities to increase production using combined interventions.

## Prepare the notebook (imports, helper functions)

In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from time import time
import os

warnings.simplefilter('ignore')
%matplotlib inline

<a id=ImportData></a>
## Import data

In [ ]:
!python --version #check the version of python installed 

In [ ]:
data_file_path = "../data/"               # set data file directory  
data_file_name = "buffalo_final_data.csv" # set data file name

In [ ]:
file_path = os.path.join(data_file_path, data_file_name) # combine and create file path 

In [ ]:
try: 
    print("Does file exists: ", os.path.exists(file_path))
    buffalo_df = pd.read_csv(file_path)
except:
    print("File load Failed. Please check if file exists..")

In [ ]:
buffalo_df.head()

In [ ]:
buffalo_df.columns

<a id=EDA></a>
## EDA
[Return to start](#TOC)

The goal of this section is to get familiar with the data that will be used for the end to end pipeline. It is very important to explore the data and summarize its main characteristics before diving in the machine learning models. It is also interesting to see how the different features are correlated with the target feature

In [ ]:
buffalo_df.info()

In [ ]:
buffalo_df.describe()

Check columns for null values

In [ ]:
null_info = buffalo_df.isna().sum()

for key in null_info.keys():
    if(null_info[key] > 0):
        print(key,":",null_info[key])

Fill missing values

In [ ]:
buffalo_df.fillna(value=0, inplace=True)

Reset index to state name

In [ ]:
buffalo_df.set_index(["State_name"], drop=True, inplace=True)

In [ ]:
top_df = buffalo_df[["total_buffalo","total_male","total_female"]]
top_df.sort_values(by="total_buffalo", axis=0, ascending=False, inplace=True)
top_df = top_df.iloc[:10]
plt.figure(figsize=(10,5))
top_df.plot.bar(colormap=plt.get_cmap('summer'), width=0.75)
plt.ylabel('Buffalo count')
plt.xlabel('State')
plt.xticks(rotation=80)

In [ ]:
labels = ['Female Buffalo', 'Male Buffalo']
sizes = [np.sum(buffalo_df.total_female), np.sum(buffalo_df.total_male)]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',startangle=90)
ax1.axis('equal')
plt.show()

<a id=FeatureEng></a>
## Feature Engineering 
[Return to start](#TOC)

When conducting an end to end Machine Learning project, after exploring and preprocessing the data it is essential to think of feature engineering. It consists of creating new feature(s) based on the features that already exist in the dataset that can be useful for training the model.

**Step 1:** Calculate total number of buffalos under each category

In [ ]:
buffalo_df["Banni"] = buffalo_df["Banni_male_total"] + buffalo_df["Banni_female_total"]
buffalo_df["Bhadawari"] = buffalo_df["Bhadawari_male_total"] + buffalo_df["Bhadawari_female_total"]
buffalo_df["Chilika"] = buffalo_df["Chilika_male_total"] + buffalo_df["Chilika_female_total"]
buffalo_df["Jaffarabadi"] = buffalo_df["Jaffarabadi_male_total"] + buffalo_df["Jaffarabadi_female_total"]
buffalo_df["Kalahandi"] = buffalo_df["Kalahandi_male_total"] + buffalo_df["Kalahandi_female_total"]
buffalo_df["Marathwadi"] = buffalo_df["Marathwadi_male_total"] + buffalo_df["Marathwadi_female_total"]
buffalo_df["Mehsana"] = buffalo_df["Mehsana_male_total"] + buffalo_df["Mehsana_female_total"]
buffalo_df["Murrah"] = buffalo_df["Murrah_male_total"] + buffalo_df["Murrah_female_total"]
buffalo_df["Nagpuri"] = buffalo_df["Nagpuri_male_total"] + buffalo_df["Nagpuri_female_total"]
buffalo_df["Nili Ravi"] = buffalo_df["Nili_Ravi_male_total"] + buffalo_df["Nili_Ravi_female_total"]
buffalo_df["Non Descript"] = buffalo_df["Non_descript_male_total"] + buffalo_df["Non_descript_female_total"]
buffalo_df["Pandharpuri"] = buffalo_df["Pandharpuri_male_total"] + buffalo_df["Pandharpuri_female_total"]
buffalo_df["Surti"] = buffalo_df["Surti_male_total"] + buffalo_df["Surti_female_total"]
buffalo_df["Toda"] = buffalo_df["Toda_male_total"] + buffalo_df["Toda_female_total"]

In [ ]:
count_df = buffalo_df[["total_buffalo","Banni","Bhadawari","Chilika","Jaffarabadi","Kalahandi","Marathwadi","Mehsana",
                       "Murrah","Nagpuri","Nili Ravi","Non Descript","Pandharpuri","Surti","Toda","total_male",
                      "total_female"]]
count_df.sort_values(by="total_buffalo", axis=0, ascending=False, inplace=True)
count_df.head()

In [ ]:
count_df.sum(axis=0)

**Step 2:** Calculate average of below fields of all past 6 years data points
- **Average in milk** - Average number of In-Milk animals per state (figures in 000 nos)
- **Average yield per In-Milk animal** - Average yield per In-Milk animals per state (figures in kg/day)
- **Average milk production** - Average milk production per state (figures in 000 tones)

In [ ]:
buffalo_df["avg_in_milk"] = (buffalo_df["201314_in_milk"] + buffalo_df["201415_in_milk"] + buffalo_df["201516_in_milk"] + 
                             buffalo_df["201617_in_milk"] + buffalo_df["201718_in_milk"] + buffalo_df["201819_in_milk"])/6

buffalo_df["avg_yield_in_milk"] = (buffalo_df["201314_yield_in_milk"] + buffalo_df["201415_yield_in_milk"] + buffalo_df["201516_yield_in_milk"] + 
                                   buffalo_df["201617_yield_in_milk"] + buffalo_df["201718_yield_in_milk"] + buffalo_df["201819_yield_in_milk"])/6

buffalo_df["avg_milk_production"] = (buffalo_df["201314_milk_production"] + buffalo_df["201415_milk_production"] + buffalo_df["201516_milk_production"] + 
                                     buffalo_df["201617_milk_production"] + buffalo_df["201718_milk_production"] + buffalo_df["201819_milk_production"])/6

**Step 3:** Round off the decimal values to 2 decimal places

In [ ]:
buffalo_df = buffalo_df.round(2)

**Step 4:** Check data correlation 

In [ ]:
corr_count_df = buffalo_df.corr()

Top 5 attributes with negative co-relation

In [ ]:
corr_count_df["total_buffalo"].sort_values(ascending=False).tail(5)

Top 5 attributes with positive co-relation 

In [ ]:
corr_count_df["total_buffalo"].sort_values(ascending=False).head(5)

In [ ]:
attributes = list(corr_count_df["total_buffalo"].sort_values(ascending=False).head(5).keys())
scatter_matrix(corr_count_df[attributes], figsize=(12, 8));

In [ ]:
buffalo_df[["avg_in_milk","avg_yield_in_milk","avg_milk_production"]].head()

**Step 5:** Buffalo breed analysis of state _**Uttar Pradesh**_ and _**Punjab**_

Uttar Pradesh state is the top most state in _**Milk Production**_ and total number of buffalo breeds in India. Below are major three types of buffalo breeds.This state has highest number of *Murrah buffalo* (known worldwide for high yield), Non Descript and Bhadawari.

In [ ]:
top_state_df = buffalo_df.loc[buffalo_df.total_buffalo == np.max(buffalo_df.total_buffalo)]
labels1 = ["Bhadawari","Murrah","Non Descript"]
sizes1 = [1583720,20110852,8772357]
top_state_df[["Banni","Bhadawari","Chilika","Jaffarabadi","Kalahandi","Marathwadi","Mehsana",
              "Murrah","Nagpuri","Nili Ravi","Non Descript","Pandharpuri","Surti","Toda"]]


Punjab stands in list of top 10 states with total number of buffalos. Below data shows that Punjab stands number one state in the list of _**Average Yield per In-Milk Animal**_ becuase of its top share of Murrah buffalo breed which is known for milk production

In [ ]:
top_state_df = buffalo_df.loc[buffalo_df.index == "PUNJAB"]
labels2 = ["Murrah","Nili Ravi","Non Descript"]
sizes2 = [4116508,485940,523218]
top_state_df[["Banni","Bhadawari","Chilika","Jaffarabadi","Kalahandi","Marathwadi","Mehsana",
              "Murrah","Nagpuri","Nili Ravi","Non Descript","Pandharpuri","Surti","Toda"]]

In [ ]:
fig, (ax0,ax1) = plt.subplots(1, 2)
fig.suptitle('Top 3 Buffalo breeds')
ax0.set_title('Uttar pradesh')
ax1.set_title('Punjab')
ax0.pie(sizes1, labels=labels1, autopct='%1.1f%%',startangle=90)
ax1.pie(sizes2, labels=labels2, autopct='%1.1f%%',startangle=90)
plt.show()

In [ ]:
Murrah_df = buffalo_df[["Murrah", "total_buffalo","avg_yield_in_milk","avg_milk_production"]]
Murrah_df["Murrah_pct_share"] = np.round(((Murrah_df.Murrah / Murrah_df.total_buffalo) * 100),2)
Murrah_df.sort_values(by="Murrah", axis=0, ascending=False, inplace=True)
Murrah_df.head(10)

<a id=Modeling></a>
## Modelling
[Return to start](#TOC)

Now that we have explored the data, cleaned it, preprocessed it and added a new feature to it, we can start the modelling part of the project by applying algorithms.

### Benchmarking 

### Model 1


In [ ]:
model_df1 = buffalo_df[["total_buffalo", "total_male", "total_female",
                       "avg_in_milk","avg_yield_in_milk","avg_milk_production"]]
model_df1.sort_values(by="avg_yield_in_milk", axis=0, ascending=False, inplace=True)
model_df1[["avg_yield_in_milk"]].head(10)

### Model 2
**Top 10 states records with highest Average Milk Production**

In [ ]:
model_df2 = buffalo_df[["total_buffalo", "total_male", "total_female",
                       "avg_in_milk","avg_yield_in_milk","avg_milk_production"]]
model_df2.sort_values(by="avg_milk_production", axis=0, ascending=False, inplace=True)
model_df2[["avg_milk_production"]].head(10)

In [ ]:
mean_avg_yield_in_milk_sample = np.round(buffalo_df.avg_yield_in_milk.mean(), decimals=2)
mean_avg_milk_production_sample = np.round(buffalo_df.avg_milk_production.mean(), decimals=2)

In [ ]:
print("Average Yield Per In-Milk Animal from Buffalos for whole sample:", mean_avg_yield_in_milk_sample)
print("Average Milk Production from Buffalos for whole sample         :", mean_avg_milk_production_sample)

Add the columns to both the models

In [ ]:
model_df1["mean_avg_yield_in_milk_sample"] = mean_avg_yield_in_milk_sample
model_df1["mean_avg_milk_production_sample"] = mean_avg_milk_production_sample
model_df2["mean_avg_yield_in_milk_sample"] = mean_avg_yield_in_milk_sample
model_df2["mean_avg_milk_production_sample"] = mean_avg_milk_production_sample

In [ ]:
model_df1[["avg_yield_in_milk", "mean_avg_yield_in_milk_sample"]].head(10)

In [ ]:
model_df2[["avg_milk_production","mean_avg_milk_production_sample"]].head(10)

In [ ]:
model_df1[["avg_yield_in_milk", "mean_avg_yield_in_milk_sample"]].tail(10)

In [ ]:
model_df2[["avg_milk_production","mean_avg_milk_production_sample"]].tail(10)

<a id=Evaluate></a>
## Evaluation, reporting, analysis
[Return to start](#TOC)

Evaluate our model, report our analysis and observations

In [ ]:
model_df1_top10 = model_df1.iloc[:10]
model_df2_top10 = model_df2.iloc[:10]

### Model 1

In [ ]:
xlabel = list(model_df1_top10.index)
ylabel = np.arange(0,10)
fig, ax = plt.subplots(figsize=(10, 5))
plt.bar(xlabel, model_df1_top10["avg_yield_in_milk"], width=0.75)
plt.axhline(y=mean_avg_yield_in_milk_sample,linewidth=1)
plt.xticks(rotation=80)
plt.xlabel('State')
plt.ylabel('Avg Yield Per In-Milk Animal in kg/day')
plt.show()

### Model 2

In [ ]:
xlabel = list(model_df2_top10.index)
ylabel = [0, 2000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000]
fig, ax = plt.subplots(figsize=(10, 5))
plt.bar(xlabel, model_df2_top10["avg_milk_production"], width=0.75)
plt.axhline(y=mean_avg_milk_production_sample,linewidth=1)
plt.xticks(rotation=80)
plt.xlabel('State')
plt.ylabel('Avg Milk Production in tones')
plt.show()

Above two graphs denotes, top 10 states are performing above average. There is possibility to increase the yield then the current attainable yield, evaluating the other socio-economic factors like agricultural data, population, climate factors etc. The analysis was limited to trend analysis based on the data points available in census data.